# Questão 2) Criação do Modelo Dimensional Estrela

O Fato ressaltado foi a Prova de um estudante, cujos atributos são as notas de cada uma das provas:
- NT_GER: Nota bruta da Prova;
- NT_FG: Nota bruta na formação geral;
- NT_OBJ_FG: Nota bruta na parte objetiva da formação geral; 
- NT_DIS_FG: Nota bruta na parte discursiva da formação geral;
- NT_CE: Nota bruta no componente específico; 
- NT_OBJ_CE: Nota bruta na parte objetiva do componente específico; 
- NT_DIS_CE: Nota bruta na parte discursiva do componente específico;

Foram identificadas **12 dimensões**:
- DIM_ANO: Ano de realização da prova;
- DIM_Estudante: Dados do estudante; 
- DIM_IES: Dados da Instituição de Ensino Superior de origem do estudante; 
- DIM_Curso: Dados do curso de origem do estudante; 
- DIM_LocalCurso: Dados do local do curso de origem do estudante;
- DIM_EnsinoMed: Dados do Ensino Médio do estudante;
- DIM_Presença: Dados referentes a presença do estudante nas prova;
- DIM_QuestPercepcao: Respostas do estudante ao Questionário de Percepção da Prova;
- DIM_Socioeconomica: Respostas do estudante ao Questionário do Estudante referente a informações socioeconomicas;
- DIM_QuestOportunidades: Respostas do estudante ao Questionário do Estudante referente a oportunidades oferecidas durante o curso;
- DIM_QuestProfessores: Respostas do estudante ao Questionário do Estudante referente aos professores do curso; 
- DIM_QuestDesenvolvimento: Respostas do estudante ao Questionário do Estudante referente ao seu desenvolvimento ao longo do curso.

A ferramenta usada para desenhar o Modelo Dimensional foi o [brModelo](http://www.sis4.com/brmodelo/).

<img src='DIM_ESTRELA_ENADE.png' width="800" height="800">

# Questão 3) Criação do Banco de Dados

In [1]:
# Importando bibliotecas necessárias
import pandas as pd
import numpy as np
import os 
import shutil
import sqlite3
from sqlite3 import Error

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## 3.1 Lendo os arquivos dos Microdados

In [3]:
curr_path = os.path.dirname(os.path.realpath('__file__')) + "/"
path_enade = curr_path + "enade/"
db_name = "ENADE_DIM.db"

In [4]:
def read_to_dataframe(year):
    csv_path = path_enade + year +  "/3.DADOS/"
    if year != '2017':
        csv_path = csv_path + f'microdados_enade_{year}.txt'
    else: 
        csv_path = csv_path + 'MICRODADOS_ENADE_2017.txt'
    
    return pd.read_csv(csv_path, sep=';', low_memory=False)
    

In [5]:
df_enade_2019 = read_to_dataframe('2019')

In [6]:
df_enade_2018 = read_to_dataframe('2018')

In [7]:
df_enade_2017 = read_to_dataframe('2017')

In [8]:
df_enade = pd.concat([df_enade_2017, df_enade_2018, df_enade_2019], axis=0)

In [9]:
del df_enade_2019, df_enade_2018, df_enade_2017

## 3.2 Utilitários para manipulação do Banco de Dados

In [10]:
db_name = "ENADE_DW.sqlite"

In [11]:
def new_db_conn(path, db_name):
    db_conn = None
    try:
        db_conn = sqlite3.connect(path + db_name)
    
    except Error as e:
        print(e)
        return False
    
    finally:
        if db_conn:
            db_conn.close()
    return 1

In [12]:
def verify_table(cursor, table_name):
    cursor.execute(''' SELECT name FROM sqlite_master WHERE type='table' AND name='{}' '''.format(nome_tabela))
    
    if cursor.fetchone():
        return 1
    else:
        return 0

In [13]:
def sqlite_type_conversion(type_str):
    if type_str in ["int32", "int64", "bool"]:
            sqlite_type = "integer"
            
    elif type_str == "float64":
            sqlite_type = "real"           
    else:
        sqlite_type = "text"
    
    return sqlite_type
    

In [14]:
def insert_table(df, cursor, table_name):
    table_exists = verify_table(cursor, table_name)
    if not table_exists:
        columns_list = list(df.columns)
        
        new_columns = []
        for column in columns_list:
            column = column + " " + sqlite_type_conversion(str(df[column].dtype))
            new_columns.append(column)
        
        column_names = ", ".join(new_columns)
        
        print(f"Creating table {table_name} on {db_name}", end='\r', flush=True)
        
        create_query = 'CREATE TABLE ' + table_name + ' (' + column_names + ')'
        cursor.execute('' + create_query + '')
        
        df.to_sql(table_name, db_conn, if_exists="append", index=False)
        db_conn.commit()
        
        
        print(" "*70, end="\r", flush=True)
        print(f"Successfully created table {table_name} on {db_name}.", end="\n\n")
    else:
        print(f"Table {table_name} already existis on {db_name}",end="\n\n")

## 3.3 Limpeza dos dados

### 3.3.1 Remoção de colunas não utilizadas

In [15]:
df_enade.drop(columns=['NU_ITEM_OFG', 'NU_ITEM_OFG_Z', 'NU_ITEM_OFG_X', 'NU_ITEM_OFG_N', 'NU_ITEM_OCE', 
                       'NU_ITEM_OCE_Z', 'NU_ITEM_OCE_X', 'NU_ITEM_OCE_N', 'DS_VT_GAB_OFG_ORIG','DS_VT_GAB_OFG_FIN',
                       'DS_VT_GAB_OCE_ORIG','DS_VT_GAB_OCE_FIN','DS_VT_ESC_OFG','DS_VT_ACE_OFG','DS_VT_ESC_OCE',         
                       'DS_VT_ACE_OCE'], inplace=True)

### 3.3.2 Limpeza de nulos

In [16]:
# Questionário de Percepção da Prova: Valores vazios ou NaN podem ser preenchidos com o caractere '.' que indica "Sem resposta"
columns = ["CO_RS_I1", "CO_RS_I2", "CO_RS_I3", "CO_RS_I4", "CO_RS_I5", "CO_RS_I6", "CO_RS_I7", "CO_RS_I8", "CO_RS_I9"]
print("Columns value counts before treatment:")
df_enade[columns].apply(pd.Series.value_counts)

Columns value counts before treatment:


,CO_RS_I1,CO_RS_I2,CO_RS_I3,CO_RS_I4,CO_RS_I5,CO_RS_I6,CO_RS_I7,CO_RS_I8,CO_RS_I9
,43606,43606,43606,43606,43606,43606,43606,43606,43606
*,596,631,1268,617,986,1354,2869,1624,1260
.,146269,146845,147261,148022,147807,148326,150391,151912,156768
A,19927,10207,192321,258875,268892,66454,201392,51871,11756
B,97945,60689,288315,650776,682924,370582,586734,131958,154877
C,712741,667636,662219,186416,176576,593982,70787,194099,365143
D,353933,439307,77681,130251,100702,194112,170437,740683,591154
E,58720,64816,21066,15174,12244,15321,207521,117984,109173


In [17]:
for column in columns:
    df_enade[column].fillna('.', inplace=True)
    df_enade[column] = np.where(df_enade[column] == ' ', '.', df_enade[column])
print("Columns with NaN after treatment:")
print(df_enade[columns].isna().sum())
print("Columns with '.' after treatment:")
df_enade[columns].apply(pd.Series.value_counts)

Columns with NaN after treatment:
CO_RS_I1    0
CO_RS_I2    0
CO_RS_I3    0
CO_RS_I4    0
CO_RS_I5    0
CO_RS_I6    0
CO_RS_I7    0
CO_RS_I8    0
CO_RS_I9    0
dtype: int64
Columns with '.' after treatment:


,CO_RS_I1,CO_RS_I2,CO_RS_I3,CO_RS_I4,CO_RS_I5,CO_RS_I6,CO_RS_I7,CO_RS_I8,CO_RS_I9
*,596,631,1268,617,986,1354,2869,1624,1260
.,275631,276207,276623,277384,277169,277688,279753,281274,286130
A,19927,10207,192321,258875,268892,66454,201392,51871,11756
B,97945,60689,288315,650776,682924,370582,586734,131958,154877
C,712741,667636,662219,186416,176576,593982,70787,194099,365143
D,353933,439307,77681,130251,100702,194112,170437,740683,591154
E,58720,64816,21066,15174,12244,15321,207521,117984,109173


In [18]:
pd.set_option('display.max_rows', None)
df_enade.dtypes

NU_ANO                  int64
CO_IES                  int64
CO_CATEGAD              int64
CO_ORGACAD              int64
CO_GRUPO                int64
CO_CURSO                int64
CO_MODALIDADE           int64
CO_MUNIC_CURSO          int64
CO_UF_CURSO             int64
CO_REGIAO_CURSO         int64
NU_IDADE                int64
TP_SEXO                object
ANO_FIM_EM            float64
ANO_IN_GRAD           float64
CO_TURNO_GRADUACAO    float64
TP_INSCRICAO_ADM        int64
TP_INSCRICAO            int64
TP_PRES               float64
TP_PR_GER             float64
TP_PR_OB_FG           float64
TP_PR_DI_FG           float64
TP_PR_OB_CE           float64
TP_PR_DI_CE           float64
TP_SFG_D1             float64
TP_SFG_D2             float64
TP_SCE_D1             float64
TP_SCE_D2             float64
TP_SCE_D3             float64
NT_GER                 object
NT_FG                  object
NT_OBJ_FG              object
NT_DIS_FG              object
NT_FG_D1               object
NT_FG_D1_P